

#### Import standard Python modules

In [113]:
#Imports de python

import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib

#### Import Azure ML SDK modules

In [114]:
#Imports de Azure

import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

#### Check Azure ML SDK version

In [115]:
# Revisar cual es la versión de Azure

print(azureml.core.VERSION)

1.0.21


#### Create Azure ML Workspace

In [116]:
# Conectarse al Workspace propio

AZ_SUBSCRIPTION_ID='1e63958c-c60b-4d54-b45d-1d3326ee4345'
ws = Workspace.create(name='pruebaRF',
                      subscription_id=AZ_SUBSCRIPTION_ID, 
                      resource_group='CognitiveEY',
                      create_resource_group=False,
                      location='southeastasia'
                     )

Deploying ContainerRegistry with name pruebarfacrfwcbtdtd.
Deployed ContainerRegistry with name pruebarfacrfwcbtdtd.
Deploying KeyVault with name pruebarfkeyvaultwxsdnjuw.
Deploying StorageAccount with name pruebarfstoragentzaurfi.
Deploying AppInsights with name pruebarfinsightsqcjtqefg.
Deployed AppInsights with name pruebarfinsightsqcjtqefg.
Deployed KeyVault with name pruebarfkeyvaultwxsdnjuw.
Deployed StorageAccount with name pruebarfstoragentzaurfi.
Deploying Workspace with name pruebaRF.
Deployed Workspace with name pruebaRF.


#### Write configuration to local file

In [118]:
# Archivos de configuracion de los servicios

ws.write_config()

Wrote the config file config.json to: C:\Users\Daniel.Roca\Documents\1-Clientes\Hack-a-lot MS\Python Notebooks\aml_config\config.json


#### Create Azure ML Experiment

In [119]:
# Crear el experimento en el workspace de Azure

exp = Experiment(workspace=ws, name='RForest')

#### Start logging metrics

In [120]:
# Ingresar en el log del experimento el tiempo de ahorita

run = exp.start_logging()                   
run.log("Experiment start time", str(datetime.datetime.now()))

#### Load salary dataset

In [121]:
# Descargarse el archivo fuente de Azure guardarlo local y manipularlo desde python.

from azure.storage.blob import BlockBlobService

block_blob_service = BlockBlobService(account_name='hackatonms', account_key='lAITAy0zk9vDkuxhYFhmA7oleq08sFmuTSaGWV/B7dBGW2l/HO9rvf5rB/DHzprLkvbYITHcta4rteJbxP4MpA==')

arch = block_blob_service.get_blob_to_path('fuentes', 'SECOP II - Contratos Demo.csv', '../Pruebadoc.csv')

file = pd.read_csv('../Pruebadoc.csv',sep=",")


#### Split the train and test data

In [122]:
file.head()

,Nombre Entidad,Nit Entidad,Proceso de Compra,Descripción del Proceso,Tipo de Contrato,Referencia del Contrato,Fecha de Inicio del Contrato,Fecha de Fin del Contrato,Fecha de Inicio de Ejecucion,Fecha de Fin de Ejecucion,...,Tipo de Proceso,Fecha de Firma,Estado BPIN,Anno BPIN,Codigo BPIN,Tipo Documento Proveedor,Documento Proveedor,Es Post Conflicto,URL Proceso,ID Contrato
0,SECRETARIA DISTRITAL DE INTEGRACION SOCIAL,899999061,CO1.BDOS.443537,PRESTACIÓN DE SERVICIOS COMO AUXILIAR ADMINIST...,Prestación de servicios,5165-2018,1,10,1,10,...,Contratación directa,07/05/2018 05:07,No Válido,2018.0,NaN,Cédula de Ciudadanía,1016039177,NaN,https://community.secop.gov.co/Public/Tenderin...,CO1.PCCNTR.453169
1,DEPARTAMENTO DE POLICIA URABA,800141103,CO1.BDOS.394774,MANTENIMIENTO PREVENTIVO Y CORRECTIVO Y/O MEJ...,Obra,42-6-10015-18,2,11,2,21,...,Selección Abreviada de Menor Cuantía,06/13/2018 02:06:47 PM,No Válido,NaN,NaN,NIT,900964257,0.0,https://community.secop.gov.co/Public/Tenderin...,CO1.PCCNTR.424348
2,GOBERNACIÓN DE SANTANDER,890201235,CO1.BDOS.253107,PRESTAR SERVICIOS DE APOYO A LA GESTIÓN ADMINI...,Prestación de servicios,CO1.PCCNTR.241016,3,12,3,12,...,Contratación Directa (con ofertas),02/07/2018 20:02,No Válido,2018.0,NaN,Cédula de Ciudadanía,1098666972,NaN,https://community.secop.gov.co/Public/Tenderin...,CO1.PCCNTR.241016
3,FONDO DE TECNOLOGÍAS DE LA INFORMACIÓN Y LAS C...,800131648,CO1.BDOS.285519,Prestar sus servicios profesionales en el trám...,Prestación de servicios,CO1.PCCNTR.277150,4,13,4,22,...,Contratación directa,01/11/2018 22:01,No Válido,2019.0,NaN,Cédula de Ciudadanía,1014197389,0.0,https://community.secop.gov.co/Public/Tenderin...,CO1.PCCNTR.277150
4,SUBRED INTEGRADA DE SERVICIOS DE SALUD CENTRO ...,900959051,CO1.BDOS.288455,PRESTAR SUS SERVICIOS DE APOYO DE MANERA PER...,Prestación de servicios,PS 1821 2018,5,14,5,14,...,Contratación régimen especial,03/28/2018 01:03:17 PM,No Válido,NaN,NaN,Cédula de Ciudadanía,39553719,NaN,https://community.secop.gov.co/Public/Tenderin...,CO1.PCCNTR.278208


In [123]:
# Crear función para calcular variable objetivo de atrasos

def calcular_atraso (df):
    if (df['Fecha de Fin de Ejecucion']-df['Fecha de Inicio del Contrato'])/(df['Fecha de Fin del Contrato']-df['Fecha de Inicio del Contrato'])>1.2:
        return 1
    return 0

# Agregar dentro del dataset la columna Atraso
    
file['Atraso'] = file.apply(lambda file: calcular_atraso(file), axis=1)     



In [124]:
# Quitar columnas que no se van a utilizar para el entrenamiento del modelo

file = file.drop('Nombre Entidad', axis = 1)
file = file.drop('Proceso de Compra', axis = 1)
file = file.drop('Descripción del Proceso', axis = 1)
file = file.drop('Referencia del Contrato', axis = 1)
file = file.drop('Fecha de Inicio del Contrato', axis = 1)
file = file.drop('Fecha de Fin del Contrato', axis = 1)
file = file.drop('Fecha de Inicio de Ejecucion', axis = 1)
file = file.drop('Fecha de Fin de Ejecucion', axis = 1)
file = file.drop('Condiciones de Entrega', axis = 1)
file = file.drop('Proveedor Adjudicado', axis = 1)
file = file.drop('Es Grupo', axis = 1)
file = file.drop('Estado Contrato', axis = 1)
file = file.drop('Habilita Pago Adelantado', axis = 1)
file = file.drop('Liquidacion', axis = 1)
file = file.drop('Obligaciones Ambientales', axis = 1)
file = file.drop('Obligaciones Postconsumo', axis = 1)
file = file.drop('Reversion', axis = 1)
file = file.drop('Valor de pago adelantado', axis = 1)
file = file.drop('Valor Facturado', axis = 1)
file = file.drop('Valor Pendiente de Pago', axis = 1)
file = file.drop('Valor Pagado', axis = 1)
file = file.drop('Valor Amortizado', axis = 1)
file = file.drop('Valor Pendiente de Amortizacion', axis = 1)
file = file.drop('Valor Pendiente de Ejecucion', axis = 1)
file = file.drop('Codigo de Categoria Principal', axis = 1)
file = file.drop('Tipo de Proceso', axis = 1)
file = file.drop('Fecha de Firma', axis = 1)
file = file.drop('Estado BPIN', axis = 1)
file = file.drop('Anno BPIN', axis = 1)
file = file.drop('Codigo BPIN', axis = 1)
file = file.drop('Tipo Documento Proveedor', axis = 1)
file = file.drop('Documento Proveedor', axis = 1)
file = file.drop('Es Post Conflicto', axis = 1)
file = file.drop('URL Proceso', axis = 1)
file = file.drop('ID Contrato', axis = 1)

file.head()

,Nit Entidad,Tipo de Contrato,Valor del Contrato,Atraso
0,899999061,Prestación de servicios,14214000,0
1,800141103,Obra,32254381,1
2,890201235,Prestación de servicios,23500000,0
3,800131648,Prestación de servicios,27192000,1
4,900959051,Prestación de servicios,15074790,0


In [125]:
# Volver las columnas categóricas dummys para el modelo

file = pd.get_dummies(file,prefix=['Tipo de Contrato'])

file.head()

,Nit Entidad,Valor del Contrato,Atraso,Tipo de Contrato_Obra,Tipo de Contrato_Prestación de servicios
0,899999061,14214000,0,0,1
1,800141103,32254381,1,1,0
2,890201235,23500000,0,0,1
3,800131648,27192000,1,0,1
4,900959051,15074790,0,0,1


In [126]:
# Dividir el dataset para utilizar el random forest

pred = np.array(file['Atraso'])
file = file.drop('Atraso', axis = 1)

lista_variables = list(file.columns)

features = np.array(file)

# importar libreria para hacer split de datos y generar set de entrenamiento 
from sklearn.model_selection import train_test_split


In [127]:
# para el split se definen las raices y el tamaño de prueba
seed = 42
size = 0.5

# Hacer el split de los datos: entrenamiento y prueba
train_features, test_features, train_labels, test_labels = train_test_split(features, pred, test_size = size, random_state = seed)

# Imprimir el tamaño de la data
print('Datos de entrenamiento', train_features.shape)
print('Encabezados de entrenamiento', train_labels.shape)
print('Datos de prueba', test_features.shape)
print('Encabezados de prueba', test_labels.shape)

Datos de entrenamiento (3, 4)
Encabezados de entrenamiento (3,)
Datos de prueba (4, 4)
Encabezados de prueba (4,)


#### Train the model

In [128]:
# Importar el modelo que se va a utilizar, en este caso Random Forest
from sklearn.ensemble import RandomForestClassifier

# Instanciación del modelo con 1000 árboles de decisión
rf = RandomForestClassifier(n_estimators = 1000, random_state = seed)

# Entrenar el modelo con los datos de entrenamiento
rf.fit(train_features, train_labels);

#### Freeze the model

In [129]:
# Guardar el modelo en un archivo local

filename = '../modelo_secop.pkl'
joblib.dump(rf, filename)

['../modelo_secop.pkl']

#### Test the model

In [130]:
# Probar el modelo con datos nuevos

filename = '../modelo_secop.pkl'
loaded_model=joblib.load(filename)
atr=loaded_model.predict_proba([[1234567,1234567,0,1]])[0]
print(atr)

[0.643 0.357]


#### Log metrics to Azure ML Experiment

In [13]:
#run.log('Intercept :', lm.intercept_)
#run.log('Slope :', lm.coef_[0])

#### End Azure ML Experiment

In [131]:
run.log("Experiment end time", str(datetime.datetime.now()))
run.complete()

#### Get Portal URL

In [132]:
# URL con la información de la corrida

print(run.get_portal_url())

https://mlworkspace.azure.ai/portal/subscriptions/1e63958c-c60b-4d54-b45d-1d3326ee4345/resourceGroups/CognitiveEY/providers/Microsoft.MachineLearningServices/workspaces/pruebaRF/experiments/RForest/runs/07429ba3-f238-452a-949f-a03cc9a568e8


#### Register the model

In [133]:
# Registro del modelo creado con Azure

model = Model.register(model_path = "../modelo_secop.pkl",
                       model_name = "modelo_secop",
                       tags = {"key": "1"},
                       description = "Predicción de Contratos",
                       workspace = ws)

Registering model modelo_secop


#### Define Azure ML Deploymemt configuration

In [134]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Contratos",  "method" : "Random Forest"}, 
                                               description='Predecir atrasos en contratos')

#### Create enviroment configuration file

In [135]:
secenv = CondaDependencies()
secenv.add_conda_package("scikit-learn")

with open("secenv.yml","w") as f:
    f.write(secenv.serialize_to_string())
with open("secenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults
- scikit-learn



#### Create Azure ML Scoring file

In [136]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('modelo_secop')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    proba_atraso = model.predict_proba(data)
    return json.dumps(proba_atraso.tolist())

Writing score.py


#### Deploy the model to Azure Container Instance


In [137]:
%%time
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="secenv.yml")

Wall time: 1 ms


#### Expose web service

In [138]:
service = Webservice.deploy_from_model(workspace=ws,
                                       name='secop-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image secop-svc:1, operation "Succeeded"
Creating service
Running.................
SucceededACI service creation operation finished, operation "Succeeded"


#### Get the Web Service URL

In [139]:
print(service.scoring_uri)

http://40.65.169.92:80/score


In [141]:
!curl -X POST \
	-H 'Content-Type:application/json' \
	-d '{"data":[[45]]}' \
	http://40.65.169.92:80/score

{"message": "Expects Content-Type to be application/json"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    73  100    58  100    15     80     20 --:--:-- --:--:-- --:--:--   108


#### Delete Workspace and clean up resources

In [86]:
ws.delete()